# Часть 1. Линейная регрессия

Датасет: https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset

Для приведенного выше датасета, построить модель линейной регрессии для
оценивания стоимости аренды недвижимости. Для решения задачи предлагается
использовать следующие подходы:

1. Получить решение задачи в замкнутом виде с помощью псевдообратной
матрицы. Рассмотреть случаи без регуляризации и с L регуляризацией.

2. Получить решение задачи методом градиентного спуска. Получить формулу
для градиента, используя матрицу признаков. Сравнить полученные решение с
п.1.

При решении задачи необходимо ответить на следующие вопросы:

1. Какие признаки оказывают наибольший вклад в точность определения
стоимости аренды? Предложить способы отбора наиболее важных признаков

2. Какая модель имеет наименьшее значение функции потерь на тестовой
выборке? Помогает ли регуляризация избежать эффекта переобучения в
данном примере?

Задание выполнил: Никита Пархоменко

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Смысл признаков:**

BHK: Number of Bedrooms, Hall, Kitchen.

Rent: Rent of the Houses/Apartments/Flats.

Size: Size of the Houses/Apartments/Flats in Square Feet.

Floor: Houses/Apartments/Flats situated in which Floor and Total Number of Floors (Example: Ground out of 2, 3 out of 5, etc.)

Area Type: Size of the Houses/Apartments/Flats calculated on either Super Area or Carpet Area or Build Area.

Area Locality: Locality of the Houses/Apartments/Flats.

City: City where the Houses/Apartments/Flats are Located.

Furnishing Status: Furnishing Status of the Houses/Apartments/Flats, either it is Furnished or Semi-Furnished or Unfurnished.

Tenant Preferred: Type of Tenant Preferred by the Owner or Agent.

Bathroom: Number of Bathrooms.

Point of Contact: Whom should you contact for more information regarding the Houses/Apartments/Flats.

---

Task: **Regression**

Target: **Rent**

## Решение

In [ ]:
df = pd.read_csv('gdrive/My Drive/ml_task_hse/task1_data/House_Rent_Dataset.csv', sep=',')
df

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [ ]:
df.describe()

,BHK,Rent,Size,Bathroom
count,4746.000000,4.746000e+03,4746.000000,4746.000000
mean,2.083860,3.499345e+04,967.490729,1.965866
std,0.832256,7.810641e+04,634.202328,0.884532
min,1.000000,1.200000e+03,10.000000,1.000000
25%,2.000000,1.000000e+04,550.000000,1.000000
50%,2.000000,1.600000e+04,850.000000,2.000000
75%,3.000000,3.300000e+04,1200.000000,2.000000
max,6.000000,3.500000e+06,8000.000000,10.000000


In [ ]:
target = 'Rent'
features = sorted(list(set(set(df.columns) - set([target]))))

numerical_features = sorted(['BHK', 'Size', 'Bathroom'])
categorical_features = sorted(list(set(set(features) - set(numerical_features))))

In [ ]:
print(f'Target: {target}')
print(f'Features: {features}\n')
print(f'Numerical features: {numerical_features}')
print(f'Categorical features: {categorical_features}')

Target: Rent
Features: ['Area Locality', 'Area Type', 'BHK', 'Bathroom', 'City', 'Floor', 'Furnishing Status', 'Point of Contact', 'Posted On', 'Size', 'Tenant Preferred']

Numerical features: ['BHK', 'Bathroom', 'Size']
Categorical features: ['Area Locality', 'Area Type', 'City', 'Floor', 'Furnishing Status', 'Point of Contact', 'Posted On', 'Tenant Preferred']


In [ ]:
print("Number of unique categories\n")

for feature in categorical_features:
  print(f"{feature}: {df[feature].nunique()}")

Number of unique categories

Area Locality: 2235
Area Type: 3
City: 6
Floor: 480
Furnishing Status: 3
Point of Contact: 3
Posted On: 81
Tenant Preferred: 3


Есть несколько фичей, принимающих слишком большое количество значений. Рассмотрим их детальнее.

### Area Locality

Это текстовое описание расположения квартиры. Теоретически, эту информацию можно было использовать для решения задачи (например, с помощью методов NLP), но не в рамках простой линейной регрессии.

Этот признак можно убрать.

In [ ]:
df['Area Locality'].head()

0                      Bandel
1    Phool Bagan, Kankurgachi
2     Salt Lake City Sector 2
3                 Dumdum Park
4               South Dum Dum
Name: Area Locality, dtype: object

### Floor

Описание этажа. Видим определенную закономерность в описаниях, теоретически отсюда можно достать две фичи - на каком этаже квартира и сколько всего этажей в доме.

Для упрощения задачи не будем учитывать эту информацию и уберем признак.

In [ ]:
df['Floor'].head()

0    Ground out of 2
1         1 out of 3
2         1 out of 3
3         1 out of 2
4         1 out of 2
Name: Floor, dtype: object

### Posted On

Это дата размещения объявления (строго говоря, не является категориальным признаком в данном случае). Мы в данном случае не работаем с последовательными данными, эта информация не будет полезной при решении задачи. Признак можно убрать.

In [ ]:
df['Posted On'].head()

0    2022-05-18
1    2022-05-13
2    2022-05-16
3    2022-07-04
4    2022-05-09
Name: Posted On, dtype: object

Уберем эти три фичи и получим новый список используемых при решении задачи фичей.

In [ ]:
eliminated_features = ['Area Locality', 'Floor', 'Posted On']

features = sorted(list(set(set(features) - set(eliminated_features))))
categorical_features = sorted(list(set(set(categorical_features) - set(eliminated_features))))

In [ ]:
print(f'Target: {target}')
print(f'Features: {features}\n')
print(f'Numerical features: {numerical_features}')
print(f'Categorical features: {categorical_features}')

Target: Rent
Features: ['Area Type', 'BHK', 'Bathroom', 'City', 'Furnishing Status', 'Point of Contact', 'Size', 'Tenant Preferred']

Numerical features: ['BHK', 'Bathroom', 'Size']
Categorical features: ['Area Type', 'City', 'Furnishing Status', 'Point of Contact', 'Tenant Preferred']


In [ ]:
print("Number of unique categories\n")

for feature in categorical_features:
  print(f"{feature}: {df[feature].nunique()}")

Number of unique categories

Area Type: 3
City: 6
Furnishing Status: 3
Point of Contact: 3
Tenant Preferred: 3


#### Сделаем пайплайн подготовки данных для решения задачи.

Отметим, что для обработки категориальных фичей в рамках линейной регрессии полезно сделать из одной категориальной фичи N бинарных фичей, где N - количество категорий.

Так мы и сделаем.

In [ ]:
from typing import List

def preprocess_data(df: pd.DataFrame, target: str, features: List, categorical_features: List):
  X, y = df[[*features]], df[target]
  X = pd.get_dummies(X, columns=categorical_features)
  final_features = list(X.columns)
  return X, y, final_features

In [ ]:
X, y, final_features = preprocess_data(df, target, features, categorical_features)
X

,BHK,Bathroom,Size,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family
0,2,2,1100,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
1,2,1,800,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
2,2,1,1000,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,2,1,800,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,2,1,850,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,2,1000,0,1,0,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0
4742,3,3,2000,0,0,1,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0
4743,3,3,1750,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4744,3,2,1500,0,1,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1


In [ ]:
print(*final_features, sep=', ')

BHK, Bathroom, Size, Area Type_Built Area, Area Type_Carpet Area, Area Type_Super Area, City_Bangalore, City_Chennai, City_Delhi, City_Hyderabad, City_Kolkata, City_Mumbai, Furnishing Status_Furnished, Furnishing Status_Semi-Furnished, Furnishing Status_Unfurnished, Point of Contact_Contact Agent, Point of Contact_Contact Builder, Point of Contact_Contact Owner, Tenant Preferred_Bachelors, Tenant Preferred_Bachelors/Family, Tenant Preferred_Family


In [ ]:
from sklearn.preprocessing import StandardScaler

# Train/test split, data scaling
def train_data_pipeline(df: pd.DataFrame,
                       target: str,
                       features: List,
                       categorical_features: List,
                       test_size: float = 0.2,
                       random_state: int = 42):

  X, y, _ = preprocess_data(df, target, features, categorical_features)
  X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=test_size, random_state=random_state)
  y_train= y_train.values.reshape(-1,1)
  y_test= y_test.values.reshape(-1,1)

  # Scaling the data
  X_scaler = StandardScaler()
  X_train = X_scaler.fit_transform(X_train)
  X_test = X_scaler.transform(X_test)
  return X_train, X_test, y_train, y_test

### 1. Получить решение задачи в замкнутом виде с помощью псевдообратной матрицы. Рассмотреть случаи без регуляризации и с L регуляризацией.

Реализуем функцию для аналитического решения задачи линейной регрессии в замкнутом виде с помощью псевдообратной матрицы.

Отметим, что для корректного решения нужного будет также вычислить значение свободного члена (intercept) в линейной регрессии. Для этого на этапе нахождения решения добавим к матрице X столбец, состоящий из единиц - это стандартная практика.

Нормальное уравнение линейной регрессии:

$w = (X^T X)^{-1} X^T y$

Нормальное уравнение линейной регрессии с L2-регуляризацией:

$w = (X^T X + α I)^{-1} X^T y$

где $α$ - коэффициент регуляризации, $I$ - единичная матрица.

In [ ]:
X_train, X_test, y_train, y_test = train_data_pipeline(df, target, features, categorical_features)

In [ ]:
from typing import Tuple

def lr_analytical_solution(X: np.array, y: np.array, stable=False, alpha=None):
  intercept_col = np.ones((len(X), 1))
  X = np.hstack((X, intercept_col))

  # Вычисление псевдообратной матрицы по формуле (может быть численно нестабильно при det -> 0)
  X_pinv = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
  # Численно стабильное вычисление псевдообратной матрицы
  if stable:
    X_pinv = np.linalg.pinv(X)
  # Если используем L2-регуляризацию, пересчитываем псевдообратную матрицу согласно уравнению (численно стабильно при alpha > 0)
  if alpha:
    X_pinv = np.dot(np.linalg.inv(np.dot(X.T, X) + alpha * np.identity(len(X[0]))), X.T)

  # Нахождение весов через псевдообратную матрицу
  weights = np.dot(X_pinv, y).reshape(-1)

  intercept = weights[-1]
  weights = weights[:-1]
  return weights, intercept

### Аналитическое решение линейной регрессии

In [ ]:
weights, intercept = lr_analytical_solution(X_train, y_train)
print("Weights:", weights)
print("Intercept:", intercept)

Weights: [ 5.95899223e+02  1.28951725e+04  2.96913540e+04  4.27472071e+16
  9.30911635e+17  9.30943195e+17 -2.06008299e+18 -2.04812176e+18
 -1.75292315e+18 -2.00997780e+18 -1.61716456e+18 -2.10717289e+18
 -1.39108511e+17 -1.99166059e+17 -1.93968814e+17 -1.31501513e+05
 -3.98549406e+03 -1.21227122e+05 -2.62889598e+03 -3.47142222e+03
 -5.47521467e+03]
Intercept: 35151.516332982006


In [ ]:
y_test_pred = np.dot(X_test, weights) + intercept

# Evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)
mse =  mean_squared_error(y_test, y_test_pred)
rmse =  np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print('Метрики качества\n')

print('MAE:', round(mae, 3))
print('MSE:', round(mse, 3))
print('RMSE:', round(rmse, 3))

print('MAPE:', round(mape, 4))
print('R2:', round(r2, 4))

Метрики качества

MAE: 45691.965
MSE: 3579888543.613
RMSE: 59832.17
MAPE: 2.987
R2: 0.1017


### Аналитическое решение линейной регрессии (стабилизация псевдообратной матрицы)

In [ ]:
weights, intercept = lr_analytical_solution(X_train, y_train, stable=True)
print("Weights:", weights)
print("Intercept:", intercept)

Weights: [ 3077.63312586 10195.10330566 23244.58228055   190.97499168
   746.16853152  -754.91245785 -2334.84694702 -5385.49018439
   433.34812394 -8751.83706058 -2750.02075637 17615.41575592
  2332.41346447  -962.87176378  -684.06454781  2184.83474221
   471.91952593 -2200.75424359  1029.37316339   862.05581479
 -2607.69223768]
Intercept: 35151.51633298204


In [ ]:
y_test_pred = np.dot(X_test, weights) + intercept

# Evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)
mse =  mean_squared_error(y_test, y_test_pred)
rmse =  np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print('Метрики качества\n')

print('MAE:', round(mae, 3))
print('MSE:', round(mse, 3))
print('RMSE:', round(rmse, 3))

print('MAPE:', round(mape, 4))
print('R2:', round(r2, 4))

Метрики качества

MAE: 22232.63
MSE: 1911180916.202
RMSE: 43717.055
MAPE: 1.1477
R2: 0.5205


### Аналитическое решение линейной регрессии c L2-регуляризацией (Ridge-регрессия)

In [ ]:
weights, intercept = lr_analytical_solution(X_train, y_train, alpha=0.001)
print("Weights:", weights)
print("Intercept:", intercept)

Weights: [ 3077.6401739  10195.10560206 23244.56647494   190.97484684
   746.16882198  -754.91260295 -2334.84505384 -5385.48781958
   433.34623708 -8751.83267931 -2750.01999234 17615.40825927
  2332.41400636  -962.87156669  -684.06516367  2184.83755335
   471.9191719  -2200.75697174  1029.37266087   862.0556043
 -2607.69119162]
Intercept: 35151.50707283795


In [ ]:
y_test_pred = np.dot(X_test, weights) + intercept

# Evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)
mse =  mean_squared_error(y_test, y_test_pred)
rmse =  np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print('Метрики качества\n')

print('MAE:', round(mae, 3))
print('MSE:', round(mse, 3))
print('RMSE:', round(rmse, 3))

print('MAPE:', round(mape, 4))
print('R2:', round(r2, 4))

Метрики качества

MAE: 22232.626
MSE: 1911181004.73
RMSE: 43717.056
MAPE: 1.1477
R2: 0.5205


### Анализ качества моделей

Как видим, использование L2-регуляризации позволяет заметно улучшить качество модели, но в данном случае это происходит за счет стабилизации расчета псевдообратной матрицы (подобные метрики качества получаются даже при очень маленьком $α$ > 0).

**Похожего результата можно добиться и без L2-регуляризации, если рассчитывать псевдообратную матрицу с использованием np.linalg.pinv(X), а не из формулы напрямую (см. выше).**


Если говорить про функции потерь, рассмотрим MSE, для удобства при этом будем сравнивать корни из MSE (RMSE).

Аналитическое решение линейной регрессии: RMSE = **59832.17**

Аналитическое решение линейной регрессии (после стабилизации): RMSE = **43717.055**

Аналитическое решение линейной регрессии с L2-регуляризацией: RMSE = **43717.056**

Еще одним хорошим показателем превосходства модели с L2-регуляризацией (после стабилизации расчетов, а не за счет борьбы с переобучением) можно считать сильное повышение **коэффициента детерминации** - он повышается с 0.1017 до 0.5205 (аналогичный результат после стабилизации расчетов и без L2).

Если судить по метрикам при стабилизации вычисления обратной матрицы, метрики качества без L2-регуляризации и с ней практически идентичны. Это говорит о том, что значимого влияния, кроме стабилизации вычислений, в данной задаче она не приносит.

### Оценка важности признаков

Для того, чтобы оценить, какие признаки оказывают наибольший вклад в определении стоимости аренды, можно посмотреть на их веса (независимо от знака при них). Вспомним, какие веса были при признаках, сопоставим их нашим итоговым признакам (учитывая бинарные, получившиеся из категориальных) и отсортируем по убыванию модуля весового коэффициента:

In [ ]:
weights_dict = {}

for k, v in zip(final_features, weights):
  weights_dict[k] = v

weights_dict = {k: v for k, v in sorted(weights_dict.items(), key=lambda item: -abs(item[1]))}
weights_dict

{'Size': 23244.566474937325,
 'City_Mumbai': 17615.4082592711,
 'Bathroom': 10195.10560206134,
 'City_Hyderabad': -8751.832679313713,
 'City_Chennai': -5385.487819582733,
 'BHK': 3077.6401739018074,
 'City_Kolkata': -2750.0199923378113,
 'Tenant Preferred_Family': -2607.691191622652,
 'City_Bangalore': -2334.8450538373463,
 'Furnishing Status_Furnished': 2332.4140063562804,
 'Point of Contact_Contact Owner': -2200.756971735186,
 'Point of Contact_Contact Agent': 2184.8375533489157,
 'Tenant Preferred_Bachelors': 1029.3726608658549,
 'Furnishing Status_Semi-Furnished': -962.8715666884083,
 'Tenant Preferred_Bachelors/Family': 862.0556043045818,
 'Area Type_Super Area': -754.9126029548047,
 'Area Type_Carpet Area': 746.1688219812161,
 'Furnishing Status_Unfurnished': -684.0651636720174,
 'Point of Contact_Contact Builder': 471.9191719037788,
 'City_Delhi': 433.34623708360175,
 'Area Type_Built Area': 190.97484683730283}

Исходя из получившейся модели, наибольшее влияние на стоимость аренды оказывает площадь жилья, город, в котором оно находится (причем может повлиять и в сторону уменьшения стоимости), количество ванных комнат и количество комнат в целом.

Это разумный результат, согласующийся со здравым смыслом.

### 2. Получить решение задачи методом градиентного спуска. Получить формулу для градиента, используя матрицу признаков. Сравнить полученные решение с п.1.

В оставшейся части задания приведем решение той же самой задачи не аналитически, а с помощью алгоритма градиентного спуска, после чего сравним получившиеся решения.

Формулу градиента можно увидеть в коде решения.

**Ответы на вопросы по самой задаче (про качество модели и важность признаков) уже были даны выше.**


In [ ]:
# Функция для решения задачи методом классического градиентного спуска
def gradient_descent(X, y, w_init=None, eta=1e-2, eps=1e-8, max_iter=1e5, alpha=None):
  X = np.array(X)
  intercept_col = np.ones((len(X), 1))
  X = np.hstack((X, intercept_col))
  y = np.array(y)
  if not w_init:
      w_init = np.zeros(X.shape[1])
  w_init = np.array(w_init)
  weight_dist = np.inf
  iter_num = 0
  w = w_init
  w = w.reshape(-1,1)
  while (weight_dist > eps) and (iter_num < max_iter):
    gradient_step = (2/len(X)) * eta * np.dot(X.T, (np.dot(X, w) - y)) # Градиентный спуск
    w_new = w - gradient_step
    # Коррекция для L2-регуляризации
    if alpha is not None:
      w_new[:-1] = w_new[:-1] - (2/len(X)) * eta * alpha * w_new[:-1]
    weight_dist = np.linalg.norm(w_new - w)
    iter_num += 1
    w = w_new

  intercept = w[-1].reshape(-1)
  weights = w[:-1].reshape(-1)
  return weights, intercept

In [ ]:
# Функция для решения задачи методом стохастического градиентного спуска
def sgd(X, y, w_init=None, eta=1e-4, eps=1e-8, max_iter=1e5, alpha=None):
  X = np.array(X)
  intercept_col = np.ones((len(X), 1))
  X = np.hstack((X, intercept_col))
  y = np.array(y)
  if not w_init:
      w_init = np.zeros(X.shape[1])
  w_init = np.array(w_init)
  weight_dist = np.inf
  iter_num = 0
  w = w_init
  while (weight_dist > eps) and (iter_num < max_iter):
    index = np.random.randint(len(X))
    w_new = w - eta * 2 * X[index] * (np.dot(w, X[index]) - y[index]) # Стохастический градиентный спуск
    # Коррекция для L2-регуляризации
    if alpha is not None:
      w_new[:-1] = w_new[:-1] - eta * 2 * alpha * w_new[:-1]
    weight_dist = np.linalg.norm(w_new - w)
    iter_num += 1
    w = w_new

  intercept = w[-1]
  weights = w[:-1]
  return weights, intercept

Поскольку требуется использовать классический метод градиентного спуска, так и поступим. Код стохастического градиентного спуска использовать не будем, но оставим здесь.

### Простая линейная регрессия

In [ ]:
weights, intercept = gradient_descent(X_train, y_train)
print("Weights:", weights)
print("Intercept:", intercept)

Weights: [ 3077.63312425 10195.10330797 23244.58227989   190.97499168
   746.16853153  -754.91245785 -2334.84694695 -5385.49018437
   433.34812401 -8751.83706059 -2750.020756   17615.41575552
  2332.41346447  -962.87176378  -684.06454781  2184.83474223
   471.91952593 -2200.75424361  1029.37316336   862.0558148
 -2607.69223767]
Intercept: [35151.51633298]


In [ ]:
y_test_pred = np.dot(X_test, weights) + intercept

# Evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)
mse =  mean_squared_error(y_test, y_test_pred)
rmse =  np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print('Метрики качества\n')

print('MAE:', round(mae, 3))
print('MSE:', round(mse, 3))
print('RMSE:', round(rmse, 3))

print('MAPE:', round(mape, 4))
print('R2:', round(r2, 4))

Метрики качества

MAE: 22232.63
MSE: 1911180916.207
RMSE: 43717.055
MAPE: 1.1477
R2: 0.5205


### Линейная регрессия с L2-регуляризацией

In [ ]:
weights, intercept = gradient_descent(X_train, y_train, alpha=0.001)
print("Weights:", weights)
print("Intercept:", intercept)

Weights: [ 3077.64017254 10195.10560487 23244.56647208   190.97484353
   746.16874807  -754.91266758 -2334.84502274 -5385.48779047
   433.34626082 -8751.83265145 -2750.01996602 17615.40828541
  2332.41401266  -962.87155556  -684.06515476  2184.83751772
   471.91917051 -2200.75700616  1029.37263783   862.05557668
 -2607.69120898]
Intercept: [35151.51633298]


In [ ]:
y_test_pred = np.dot(X_test, weights) + intercept

# Evaluation metrics
mae = mean_absolute_error(y_test, y_test_pred)
mape = mean_absolute_percentage_error(y_test, y_test_pred)
mse =  mean_squared_error(y_test, y_test_pred)
rmse =  np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print('Метрики качества\n')

print('MAE:', round(mae, 3))
print('MSE:', round(mse, 3))
print('RMSE:', round(rmse, 3))

print('MAPE:', round(mape, 4))
print('R2:', round(r2, 4))

Метрики качества

MAE: 22232.627
MSE: 1911181015.564
RMSE: 43717.056
MAPE: 1.1477
R2: 0.5205


### Анализ качества моделей

Как видим, использование L2-регуляризации при реализации метода градиентного спуска в данном примере не позволяет значимо повысить качество модели и уменьшить функцию потерь.

**Эти результаты полностью согласуются с полученными при аналитическом решении задачи линейной регрессии.**